In [ ]:
import pandas as pd

df = pd.read_csv('../all_ranks.csv')
df.fillna(0, inplace=True)
data = df.to_dict('records')
df

In [ ]:
from qdrant_client import QdrantClient, models
from sentence_transformers import SentenceTransformer

encoder = SentenceTransformer('all-mpnet-base-v2')

qdrant = QdrantClient(":memory:")

def create_collection(collection_name):
    return qdrant.create_collection("all_ranks",
                               vectors_config=models.VectorParams(
                                   size=encoder.get_sentence_embedding_dimension(),
                                   distance=models.Distance.COSINE
                                   )
                                   )

if qdrant.collection_exists("all_ranks"):
    qdrant.delete_collection("all_ranks")
    collection = create_collection("all_ranks")
else:
    collection = create_collection("all_ranks")


In [ ]:
qdrant.upload_points(
    collection_name="all_ranks",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["Requirement Name"]).tolist(),
            payload=doc
        ) for idx, doc in enumerate(data)
    ]
    )

In [ ]:
results = qdrant.search(
    collection_name="all_ranks",
    query_vector=encoder.encode("fire").tolist(),
    limit=5
)

results
